* **Slugging Percentage (SLG)** - Only focusses on the player's ability to hit the ball
* **On-Base Percentage (OBP)** - Focusses on the the player's capacity to draw a walk, as well as their capacity to hit the ball

# Analyse the relationship between the player's salaries and the on-base percentage

* In a competitive market - we should expect to observe salaries to be strongly correlated with on-base percentage (OBP), more so than with slugging percentage (SLG)


* The table we are trying to reproduce run a regression of batter salaries on OBP, SLG and other factors that affect salary level, including:
    * **Plate appearances** - how often do you play?
    * **Arbitration Eligibility & Free Agency** - bargaining status relative to the owner. Rookie has no bargaining status --> After few seasons, you become Arbitration Eligible [= you can challenge salary offered. An orbitrator decides whats a fair salary you should be given, based on your skills] --> After 6 years, you become a free agent [=sell your service to the highest bidder].
    * **Fielding Position** - catchers in particular positions provide valuable service - can command higher salary for same level of batting performance.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## I. Collect Salary Data

In [2]:
pd. set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 50)

In [3]:
Salary = pd.read_csv("datasets/Salaries.csv")
display(Salary)

,yearID,teamID,lgID,playerID,salary
0,1985,ATL,NL,barkele01,870000
1,1985,ATL,NL,bedrost01,550000
2,1985,ATL,NL,benedbr01,545000
3,1985,ATL,NL,campri01,633333
4,1985,ATL,NL,ceronri01,625000
...,...,...,...,...,...
26423,2016,WAS,NL,strasst01,10400000
26424,2016,WAS,NL,taylomi02,524000
26425,2016,WAS,NL,treinbl01,524900
26426,2016,WAS,NL,werthja01,21733615


In [4]:
#Drop missing values
Salary = Salary[Salary.salary > 0]
Salary

,yearID,teamID,lgID,playerID,salary
0,1985,ATL,NL,barkele01,870000
1,1985,ATL,NL,bedrost01,550000
2,1985,ATL,NL,benedbr01,545000
3,1985,ATL,NL,campri01,633333
4,1985,ATL,NL,ceronri01,625000
...,...,...,...,...,...
26423,2016,WAS,NL,strasst01,10400000
26424,2016,WAS,NL,taylomi02,524000
26425,2016,WAS,NL,treinbl01,524900
26426,2016,WAS,NL,werthja01,21733615


### When you have a variable that ranges from very small to very large --> Use logarithm of the variable! If you don't do this, the very large numbers will be given more significance, and thus will distort the result!

In [5]:
#Calculate the log of salary
Salary["lnSal"] = np.log(Salary["salary"])
Salary

,yearID,teamID,lgID,playerID,salary,lnSal
0,1985,ATL,NL,barkele01,870000,13.676248
1,1985,ATL,NL,bedrost01,550000,13.217674
2,1985,ATL,NL,benedbr01,545000,13.208541
3,1985,ATL,NL,campri01,633333,13.358752
4,1985,ATL,NL,ceronri01,625000,13.345507
...,...,...,...,...,...,...
26423,2016,WAS,NL,strasst01,10400000,16.157316
26424,2016,WAS,NL,taylomi02,524000,13.169247
26425,2016,WAS,NL,treinbl01,524900,13.170963
26426,2016,WAS,NL,werthja01,21733615,16.894371


In [6]:
#Rename yearID column
Salary = Salary.rename(columns = {"yearID":"SalYear"})
Salary

,SalYear,teamID,lgID,playerID,salary,lnSal
0,1985,ATL,NL,barkele01,870000,13.676248
1,1985,ATL,NL,bedrost01,550000,13.217674
2,1985,ATL,NL,benedbr01,545000,13.208541
3,1985,ATL,NL,campri01,633333,13.358752
4,1985,ATL,NL,ceronri01,625000,13.345507
...,...,...,...,...,...,...
26423,2016,WAS,NL,strasst01,10400000,16.157316
26424,2016,WAS,NL,taylomi02,524000,13.169247
26425,2016,WAS,NL,treinbl01,524900,13.170963
26426,2016,WAS,NL,werthja01,21733615,16.894371


In [7]:
"""We will create a Master DF, where all the required data will be added"""
Master = Salary
Master

,SalYear,teamID,lgID,playerID,salary,lnSal
0,1985,ATL,NL,barkele01,870000,13.676248
1,1985,ATL,NL,bedrost01,550000,13.217674
2,1985,ATL,NL,benedbr01,545000,13.208541
3,1985,ATL,NL,campri01,633333,13.358752
4,1985,ATL,NL,ceronri01,625000,13.345507
...,...,...,...,...,...,...
26423,2016,WAS,NL,strasst01,10400000,16.157316
26424,2016,WAS,NL,taylomi02,524000,13.169247
26425,2016,WAS,NL,treinbl01,524900,13.170963
26426,2016,WAS,NL,werthja01,21733615,16.894371


## II. Collecting Batting Data

In [9]:
Batting = pd.read_csv("datasets/Batting.csv")
display(Batting)

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,Doubles,Triples,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,0,0,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,0,0,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,NaN
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,5,0,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,NaN
3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,2,2,27.0,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,NaN
4,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,3,0,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102811,zimmejo02,2016,1,DET,AL,19,4,0,1,0,0,0,0.0,0.0,0.0,0,2.0,0.0,0.0,1.0,0.0,0.0
102812,zimmery01,2016,1,WAS,NL,115,427,60,93,18,1,15,46.0,4.0,1.0,29,104.0,1.0,5.0,0.0,6.0,12.0
102813,zobribe01,2016,1,CHN,NL,147,523,94,142,31,3,18,76.0,6.0,4.0,96,82.0,6.0,4.0,4.0,4.0,17.0
102814,zuninmi01,2016,1,SEA,AL,55,164,16,34,7,0,12,31.0,0.0,0.0,21,65.0,0.0,6.0,0.0,1.0,0.0


* **Stint** - Created for players who played with 2 different teams in a single season - The player will have stint = 1 for the first team he played for, and stint = 2 for the team he moved to in that season.

* **We want to merge these 2 rows for players who played for 2 different teams in the season. We want to ensure that each player has only 1 row in the table, for each season**

In [10]:
Batting = Batting.groupby(['playerID', 'yearID']).sum()
Batting.reset_index(inplace = True)
display(Batting)

,playerID,yearID,stint,G,AB,R,H,Doubles,Triples,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,aardsda01,2004,1,11,0,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,aardsda01,2006,1,45,2,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0
2,aardsda01,2007,1,25,0,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,aardsda01,2008,1,47,1,0,0,0,0,0,0.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0
4,aardsda01,2009,1,73,0,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95245,zwilldu01,1914,1,154,592,91,185,38,8,16,95.0,21.0,0.0,46,68.0,0.0,1.0,10.0,0.0,0.0
95246,zwilldu01,1915,1,150,548,65,157,32,7,13,94.0,24.0,0.0,67,65.0,0.0,2.0,18.0,0.0,0.0
95247,zwilldu01,1916,1,35,53,4,6,1,0,1,8.0,0.0,0.0,4,6.0,0.0,0.0,2.0,0.0,0.0
95248,zychto01,2015,1,13,0,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0


* SLG = (Singles + 2*Doubles + 3*Triples + 4*Homeruns) / Atbats:
    * Singles = Hits - (Doubles + Triples + Homeruns)
* OBP = (Hits + Bases on Balls + Hit by Pitch) / (At bats + Bases on Balls + Hit by Pitch + Sacrifice Flies)
* Plate Appearances = At bats + Base on Balls + Hit by Pitch + Sacrifice hits + Sacrifice flies

* **Filter Required Data:**
    * Variables - AB, H, 2B, 3B, HR, R, BB, HBP, SF, SH
    * Date - 1993 to 2016
    * At bats > 130 (players with less than 130 at bats are all rookies)

In [11]:
Batting = Batting[(Batting.yearID >= 1993) & (Batting.yearID <= 2016) & (Batting.AB >= 130)].copy()
display(Batting)

,playerID,yearID,stint,G,AB,R,H,Doubles,Triples,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
98,abbotje01,1998,1,89,244,33,68,14,1,12,41.0,3.0,3.0,9,28.0,1.0,0.0,2.0,5.0,2.0
100,abbotje01,2000,1,80,215,31,59,15,1,3,29.0,2.0,1.0,21,38.0,1.0,2.0,2.0,1.0,2.0
113,abbotku01,1994,1,101,345,41,86,17,3,9,33.0,3.0,0.0,16,98.0,1.0,5.0,3.0,2.0,5.0
114,abbotku01,1995,1,120,420,60,107,18,7,17,60.0,4.0,3.0,36,110.0,4.0,5.0,2.0,5.0,6.0
115,abbotku01,1996,1,109,320,37,81,18,7,8,33.0,3.0,3.0,22,99.0,1.0,3.0,4.0,0.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95216,zuninmi01,2013,1,52,173,22,37,5,0,5,14.0,1.0,0.0,16,49.0,0.0,3.0,0.0,1.0,5.0
95217,zuninmi01,2014,1,131,438,51,87,20,2,22,60.0,0.0,3.0,17,158.0,1.0,17.0,0.0,4.0,12.0
95218,zuninmi01,2015,1,112,350,28,61,11,0,11,28.0,0.0,1.0,21,132.0,0.0,5.0,8.0,2.0,6.0
95219,zuninmi01,2016,1,55,164,16,34,7,0,12,31.0,0.0,0.0,21,65.0,0.0,6.0,0.0,1.0,0.0


### Calculate Batting Statistics:
* PA (Plate Appearances) = AB + BB + HBP + SH + SF
* OBP = (H + BB + HBP) / (AB + BB + HBP + SF)
* SLG = ((H - 2B - 3B - HR) + 2*2B + 3*3B + 4*HR) / AB

Here: BB (Walks), AB(At bats), HBP(Hit by Pitch), SH(Sacrifice Hits), SF(Sacrifice Flies), H(Hits), HR(HomeRun)

In [12]:
Batting["PA"] = Batting["AB"] + Batting["BB"] + Batting["HBP"] + Batting["SH"] + Batting["SF"]
Batting["OBP"] = (Batting["H"]+Batting["HBP"]+Batting["BB"]) / (Batting["AB"] + Batting["BB"] + Batting["HBP"] + Batting["SF"])
Batting["SLG"] = ((Batting["H"] - Batting["Doubles"] - Batting["Triples"] - Batting["HR"]) + 2*Batting["Doubles"] 
                  + 3*Batting["Triples"] + 4*Batting["HR"]) / Batting["AB"]

display(Batting)

,playerID,yearID,stint,G,AB,R,H,Doubles,Triples,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,PA,OBP,SLG
98,abbotje01,1998,1,89,244,33,68,14,1,12,41.0,3.0,3.0,9,28.0,1.0,0.0,2.0,5.0,2.0,260.0,0.298450,0.491803
100,abbotje01,2000,1,80,215,31,59,15,1,3,29.0,2.0,1.0,21,38.0,1.0,2.0,2.0,1.0,2.0,241.0,0.343096,0.395349
113,abbotku01,1994,1,101,345,41,86,17,3,9,33.0,3.0,0.0,16,98.0,1.0,5.0,3.0,2.0,5.0,371.0,0.290761,0.394203
114,abbotku01,1995,1,120,420,60,107,18,7,17,60.0,4.0,3.0,36,110.0,4.0,5.0,2.0,5.0,6.0,468.0,0.317597,0.452381
115,abbotku01,1996,1,109,320,37,81,18,7,8,33.0,3.0,3.0,22,99.0,1.0,3.0,4.0,0.0,7.0,349.0,0.307246,0.428125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95216,zuninmi01,2013,1,52,173,22,37,5,0,5,14.0,1.0,0.0,16,49.0,0.0,3.0,0.0,1.0,5.0,193.0,0.290155,0.329480
95217,zuninmi01,2014,1,131,438,51,87,20,2,22,60.0,0.0,3.0,17,158.0,1.0,17.0,0.0,4.0,12.0,476.0,0.254202,0.404110
95218,zuninmi01,2015,1,112,350,28,61,11,0,11,28.0,0.0,1.0,21,132.0,0.0,5.0,8.0,2.0,6.0,386.0,0.230159,0.300000
95219,zuninmi01,2016,1,55,164,16,34,7,0,12,31.0,0.0,0.0,21,65.0,0.0,6.0,0.0,1.0,0.0,192.0,0.317708,0.469512


# III. Merge Salary and Batting Data
**YOU NEED TO MERGE BATTING DATA OF A PLAYER, WITH THEIR SALARY DATA FOR NEXT YEAR --> OUR LOGIC IS THAT BATTING PERFORMANCE OF A PLAYER SHOULD DETERMINE THEIR SALARIES, SO BATTING STATISTICS OF LAST YEAR, SHOULD DETERMINE THE PLAYER SALARY OF THIS YEAR!**

In [13]:
Batting["SalYear"] = Batting["yearID"] + 1
display(Batting)

,playerID,yearID,stint,G,AB,R,H,Doubles,Triples,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,PA,OBP,SLG,SalYear
98,abbotje01,1998,1,89,244,33,68,14,1,12,41.0,3.0,3.0,9,28.0,1.0,0.0,2.0,5.0,2.0,260.0,0.298450,0.491803,1999
100,abbotje01,2000,1,80,215,31,59,15,1,3,29.0,2.0,1.0,21,38.0,1.0,2.0,2.0,1.0,2.0,241.0,0.343096,0.395349,2001
113,abbotku01,1994,1,101,345,41,86,17,3,9,33.0,3.0,0.0,16,98.0,1.0,5.0,3.0,2.0,5.0,371.0,0.290761,0.394203,1995
114,abbotku01,1995,1,120,420,60,107,18,7,17,60.0,4.0,3.0,36,110.0,4.0,5.0,2.0,5.0,6.0,468.0,0.317597,0.452381,1996
115,abbotku01,1996,1,109,320,37,81,18,7,8,33.0,3.0,3.0,22,99.0,1.0,3.0,4.0,0.0,7.0,349.0,0.307246,0.428125,1997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95216,zuninmi01,2013,1,52,173,22,37,5,0,5,14.0,1.0,0.0,16,49.0,0.0,3.0,0.0,1.0,5.0,193.0,0.290155,0.329480,2014
95217,zuninmi01,2014,1,131,438,51,87,20,2,22,60.0,0.0,3.0,17,158.0,1.0,17.0,0.0,4.0,12.0,476.0,0.254202,0.404110,2015
95218,zuninmi01,2015,1,112,350,28,61,11,0,11,28.0,0.0,1.0,21,132.0,0.0,5.0,8.0,2.0,6.0,386.0,0.230159,0.300000,2016
95219,zuninmi01,2016,1,55,164,16,34,7,0,12,31.0,0.0,0.0,21,65.0,0.0,6.0,0.0,1.0,0.0,192.0,0.317708,0.469512,2017


In [14]:
Master = pd.merge(Batting, Master, on = ["SalYear", "playerID"])
display(Master)
"""We have merged a player's batting performance data, with the Salary data of next year"""

,playerID,yearID,stint,G,AB,R,H,Doubles,Triples,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,PA,OBP,SLG,SalYear,teamID,lgID,salary,lnSal
0,abbotje01,1998,1,89,244,33,68,14,1,12,41.0,3.0,3.0,9,28.0,1.0,0.0,2.0,5.0,2.0,260.0,0.298450,0.491803,1999,CHA,AL,255000,12.449019
1,abbotje01,2000,1,80,215,31,59,15,1,3,29.0,2.0,1.0,21,38.0,1.0,2.0,2.0,1.0,2.0,241.0,0.343096,0.395349,2001,FLO,NL,300000,12.611538
2,abbotku01,1994,1,101,345,41,86,17,3,9,33.0,3.0,0.0,16,98.0,1.0,5.0,3.0,2.0,5.0,371.0,0.290761,0.394203,1995,FLO,NL,119000,11.686879
3,abbotku01,1995,1,120,420,60,107,18,7,17,60.0,4.0,3.0,36,110.0,4.0,5.0,2.0,5.0,6.0,468.0,0.317597,0.452381,1996,FLO,NL,250000,12.429216
4,abbotku01,1996,1,109,320,37,81,18,7,8,33.0,3.0,3.0,22,99.0,1.0,3.0,4.0,0.0,7.0,349.0,0.307246,0.428125,1997,FLO,NL,650000,13.384728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7779,zobribe01,2013,1,157,612,77,168,36,3,12,71.0,11.0,3.0,72,91.0,4.0,7.0,1.0,6.0,18.0,698.0,0.354376,0.401961,2014,TBA,AL,7000000,15.761421
7780,zobribe01,2014,1,146,570,83,155,34,3,10,52.0,10.0,5.0,75,84.0,4.0,1.0,2.0,6.0,8.0,654.0,0.354294,0.394737,2015,OAK,AL,7500000,15.830414
7781,zobribe01,2015,3,126,467,76,129,36,3,13,56.0,3.0,4.0,62,56.0,3.0,1.0,0.0,5.0,8.0,535.0,0.358879,0.449679,2016,CHN,NL,10500000,16.166886
7782,zuninmi01,2013,1,52,173,22,37,5,0,5,14.0,1.0,0.0,16,49.0,0.0,3.0,0.0,1.0,5.0,193.0,0.290155,0.329480,2014,SEA,AL,504100,13.130530


"We have merged a player's batting performance data, with the Salary data of next year"

# IV. Collecting Player Negotiation and Arbitaration Data

* Player can be:
    * **Rookie [First 2 years]** - Have no negotiating capacity.
    * **Aribitaration Eligible [After 2nd year]** - If they aren't satisfied with their salary, they can go to an individual arbitrator who will decide what is a fair salary for the player.
    * **Free Agent [After 6 years]** - Player can sell their services to the highest bidder.
    
* To determine the status of the player - identify when the player made his debut --> calculate the years elapsed to identify his status

In [15]:
People = pd.read_csv("datasets/People.csv")
display(People)

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,NaN,NaN,NaN,David,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,NaN,NaN,NaN,Hank,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,USA,GA,Atlanta,Tommie,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,NaN,NaN,NaN,Don,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,NaN,NaN,NaN,Andy,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19365,zupofr01,1939.0,8.0,29.0,USA,CA,San Francisco,2005.0,3.0,25.0,USA,CA,Burlingame,Frank,Zupo,Frank Joseph,182.0,71.0,L,R,1957-07-01,1961-05-09,zupof101,zupofr01
19366,zuvelpa01,1958.0,10.0,31.0,USA,CA,San Mateo,NaN,NaN,NaN,NaN,NaN,NaN,Paul,Zuvella,Paul,173.0,72.0,R,R,1982-09-04,1991-05-02,zuvep001,zuvelpa01
19367,zuverge01,1924.0,8.0,20.0,USA,MI,Holland,2014.0,9.0,8.0,USA,AZ,Tempe,George,Zuverink,George,195.0,76.0,R,R,1951-04-21,1959-06-15,zuveg101,zuverge01
19368,zwilldu01,1888.0,11.0,2.0,USA,MO,St. Louis,1978.0,3.0,27.0,USA,CA,La Crescenta,Dutch,Zwilling,Edward Harrison,160.0,66.0,L,L,1910-08-14,1916-07-12,zwild101,zwilldu01


In [16]:
Debut = People[["playerID", "debut"]].copy()
Debut

,playerID,debut
0,aardsda01,2004-04-06
1,aaronha01,1954-04-13
2,aaronto01,1962-04-10
3,aasedo01,1977-07-26
4,abadan01,2001-09-10
...,...,...
19365,zupofr01,1957-07-01
19366,zuvelpa01,1982-09-04
19367,zuverge01,1951-04-21
19368,zwilldu01,1910-08-14


In [17]:
Debut["debutyr"] = Debut["debut"].astype(str).str[0:4]
Debut

,playerID,debut,debutyr
0,aardsda01,2004-04-06,2004
1,aaronha01,1954-04-13,1954
2,aaronto01,1962-04-10,1962
3,aasedo01,1977-07-26,1977
4,abadan01,2001-09-10,2001
...,...,...,...
19365,zupofr01,1957-07-01,1957
19366,zuvelpa01,1982-09-04,1982
19367,zuverge01,1951-04-21,1951
19368,zwilldu01,1910-08-14,1910


In [18]:
Debut = Debut[["playerID","debutyr"]]
Debut

,playerID,debutyr
0,aardsda01,2004
1,aaronha01,1954
2,aaronto01,1962
3,aasedo01,1977
4,abadan01,2001
...,...,...
19365,zupofr01,1957
19366,zuvelpa01,1982
19367,zuverge01,1951
19368,zwilldu01,1910


In [19]:
#Merge (how = left), so that we only have player debut for the players that already exist in the list
Master = pd.merge(Master, Debut, on = ["playerID"], how = "left")
display(Master)

,playerID,yearID,stint,G,AB,R,H,Doubles,Triples,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,PA,OBP,SLG,SalYear,teamID,lgID,salary,lnSal,debutyr
0,abbotje01,1998,1,89,244,33,68,14,1,12,41.0,3.0,3.0,9,28.0,1.0,0.0,2.0,5.0,2.0,260.0,0.298450,0.491803,1999,CHA,AL,255000,12.449019,1997
1,abbotje01,2000,1,80,215,31,59,15,1,3,29.0,2.0,1.0,21,38.0,1.0,2.0,2.0,1.0,2.0,241.0,0.343096,0.395349,2001,FLO,NL,300000,12.611538,1997
2,abbotku01,1994,1,101,345,41,86,17,3,9,33.0,3.0,0.0,16,98.0,1.0,5.0,3.0,2.0,5.0,371.0,0.290761,0.394203,1995,FLO,NL,119000,11.686879,1993
3,abbotku01,1995,1,120,420,60,107,18,7,17,60.0,4.0,3.0,36,110.0,4.0,5.0,2.0,5.0,6.0,468.0,0.317597,0.452381,1996,FLO,NL,250000,12.429216,1993
4,abbotku01,1996,1,109,320,37,81,18,7,8,33.0,3.0,3.0,22,99.0,1.0,3.0,4.0,0.0,7.0,349.0,0.307246,0.428125,1997,FLO,NL,650000,13.384728,1993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7779,zobribe01,2013,1,157,612,77,168,36,3,12,71.0,11.0,3.0,72,91.0,4.0,7.0,1.0,6.0,18.0,698.0,0.354376,0.401961,2014,TBA,AL,7000000,15.761421,2006
7780,zobribe01,2014,1,146,570,83,155,34,3,10,52.0,10.0,5.0,75,84.0,4.0,1.0,2.0,6.0,8.0,654.0,0.354294,0.394737,2015,OAK,AL,7500000,15.830414,2006
7781,zobribe01,2015,3,126,467,76,129,36,3,13,56.0,3.0,4.0,62,56.0,3.0,1.0,0.0,5.0,8.0,535.0,0.358879,0.449679,2016,CHN,NL,10500000,16.166886,2006
7782,zuninmi01,2013,1,52,173,22,37,5,0,5,14.0,1.0,0.0,16,49.0,0.0,3.0,0.0,1.0,5.0,193.0,0.290155,0.329480,2014,SEA,AL,504100,13.130530,2013


In [20]:
#Calculate the years of experience for a player
Master["Exp"]=Master["yearID"]-Master["debutyr"].astype(int) #need to mention that we want the result as int (curr. its string)
display(Master)

,playerID,yearID,stint,G,AB,R,H,Doubles,Triples,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,PA,OBP,SLG,SalYear,teamID,lgID,salary,lnSal,debutyr,Exp
0,abbotje01,1998,1,89,244,33,68,14,1,12,41.0,3.0,3.0,9,28.0,1.0,0.0,2.0,5.0,2.0,260.0,0.298450,0.491803,1999,CHA,AL,255000,12.449019,1997,1
1,abbotje01,2000,1,80,215,31,59,15,1,3,29.0,2.0,1.0,21,38.0,1.0,2.0,2.0,1.0,2.0,241.0,0.343096,0.395349,2001,FLO,NL,300000,12.611538,1997,3
2,abbotku01,1994,1,101,345,41,86,17,3,9,33.0,3.0,0.0,16,98.0,1.0,5.0,3.0,2.0,5.0,371.0,0.290761,0.394203,1995,FLO,NL,119000,11.686879,1993,1
3,abbotku01,1995,1,120,420,60,107,18,7,17,60.0,4.0,3.0,36,110.0,4.0,5.0,2.0,5.0,6.0,468.0,0.317597,0.452381,1996,FLO,NL,250000,12.429216,1993,2
4,abbotku01,1996,1,109,320,37,81,18,7,8,33.0,3.0,3.0,22,99.0,1.0,3.0,4.0,0.0,7.0,349.0,0.307246,0.428125,1997,FLO,NL,650000,13.384728,1993,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7779,zobribe01,2013,1,157,612,77,168,36,3,12,71.0,11.0,3.0,72,91.0,4.0,7.0,1.0,6.0,18.0,698.0,0.354376,0.401961,2014,TBA,AL,7000000,15.761421,2006,7
7780,zobribe01,2014,1,146,570,83,155,34,3,10,52.0,10.0,5.0,75,84.0,4.0,1.0,2.0,6.0,8.0,654.0,0.354294,0.394737,2015,OAK,AL,7500000,15.830414,2006,8
7781,zobribe01,2015,3,126,467,76,129,36,3,13,56.0,3.0,4.0,62,56.0,3.0,1.0,0.0,5.0,8.0,535.0,0.358879,0.449679,2016,CHN,NL,10500000,16.166886,2006,9
7782,zuninmi01,2013,1,52,173,22,37,5,0,5,14.0,1.0,0.0,16,49.0,0.0,3.0,0.0,1.0,5.0,193.0,0.290155,0.329480,2014,SEA,AL,504100,13.130530,2013,0


### Create dummy variables for player status - Arb, Free, Rookie
**ROOKIE IS THE DEFAULT CATEGORY. THERE MUST ALWAYS BE A DEFAULT CATEGORY WHEN USING DUMMY VARIABLES IN A REGRESSION. HERE IF ARB = 0 AND FREE = 0 --> PLAYER IS A ROOKIE (BY DEFAULT)**

In [21]:
Master["Arb"] = np.where((Master["Exp"] <= 6) & (Master["Exp"] >= 3), 1, 0)
Master["Free"] = np.where(Master["Exp"] > 6, 1, 0)
Master

,playerID,yearID,stint,G,AB,R,H,Doubles,Triples,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,PA,OBP,SLG,SalYear,teamID,lgID,salary,lnSal,debutyr,Exp,Arb,Free
0,abbotje01,1998,1,89,244,33,68,14,1,12,41.0,3.0,3.0,9,28.0,1.0,0.0,2.0,5.0,2.0,260.0,0.298450,0.491803,1999,CHA,AL,255000,12.449019,1997,1,0,0
1,abbotje01,2000,1,80,215,31,59,15,1,3,29.0,2.0,1.0,21,38.0,1.0,2.0,2.0,1.0,2.0,241.0,0.343096,0.395349,2001,FLO,NL,300000,12.611538,1997,3,1,0
2,abbotku01,1994,1,101,345,41,86,17,3,9,33.0,3.0,0.0,16,98.0,1.0,5.0,3.0,2.0,5.0,371.0,0.290761,0.394203,1995,FLO,NL,119000,11.686879,1993,1,0,0
3,abbotku01,1995,1,120,420,60,107,18,7,17,60.0,4.0,3.0,36,110.0,4.0,5.0,2.0,5.0,6.0,468.0,0.317597,0.452381,1996,FLO,NL,250000,12.429216,1993,2,0,0
4,abbotku01,1996,1,109,320,37,81,18,7,8,33.0,3.0,3.0,22,99.0,1.0,3.0,4.0,0.0,7.0,349.0,0.307246,0.428125,1997,FLO,NL,650000,13.384728,1993,3,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7779,zobribe01,2013,1,157,612,77,168,36,3,12,71.0,11.0,3.0,72,91.0,4.0,7.0,1.0,6.0,18.0,698.0,0.354376,0.401961,2014,TBA,AL,7000000,15.761421,2006,7,0,1
7780,zobribe01,2014,1,146,570,83,155,34,3,10,52.0,10.0,5.0,75,84.0,4.0,1.0,2.0,6.0,8.0,654.0,0.354294,0.394737,2015,OAK,AL,7500000,15.830414,2006,8,0,1
7781,zobribe01,2015,3,126,467,76,129,36,3,13,56.0,3.0,4.0,62,56.0,3.0,1.0,0.0,5.0,8.0,535.0,0.358879,0.449679,2016,CHN,NL,10500000,16.166886,2006,9,0,1
7782,zuninmi01,2013,1,52,173,22,37,5,0,5,14.0,1.0,0.0,16,49.0,0.0,3.0,0.0,1.0,5.0,193.0,0.290155,0.329480,2014,SEA,AL,504100,13.130530,2013,0,0,0


# V. Collecting Player Fielding Data

* The dataset has the fielding positions:
    * catcher (G_c)
    * first base (G_1b)
    * second base (G_2b)
    * third base (G_3b)
    * short stop (G_ss)
    * outfileder (G_of)
    * designated hitters (G_dh) --> batters, who do not field. 

In [22]:
Appearances = pd.read_csv("datasets/Appearances.csv")
Appearances

,yearID,teamID,lgID,playerID,G_all,GS,G_batting,G_defense,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_of,G_dh,G_ph,G_pr
0,1871,TRO,NaN,abercda01,1,1.0,1,1.0,0,0,0,0,0,1,0,0,0,0,0.0,0.0,0.0
1,1871,RC1,NaN,addybo01,25,25.0,25,25.0,0,0,0,22,0,3,0,0,0,0,0.0,0.0,0.0
2,1871,CL1,NaN,allisar01,29,29.0,29,29.0,0,0,0,2,0,0,0,29,0,29,0.0,0.0,0.0
3,1871,WS3,NaN,allisdo01,27,27.0,27,27.0,0,27,0,0,0,0,0,0,0,0,0.0,0.0,0.0
4,1871,RC1,NaN,ansonca01,25,25.0,25,25.0,0,5,1,2,20,0,1,0,0,1,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104251,2017,DET,AL,zimmejo02,29,29.0,3,29.0,29,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0
104252,2017,WAS,NL,zimmery01,144,132.0,144,143.0,0,0,143,0,0,0,0,0,0,0,1.0,4.0,0.0
104253,2017,CHN,NL,zobribe01,128,105.0,128,118.0,0,0,5,81,0,5,36,0,32,61,0.0,20.0,0.0
104254,2017,SEA,AL,zuninmi01,124,112.0,124,120.0,0,120,0,0,0,0,0,0,0,0,2.0,4.0,1.0


In [23]:
#This dataset also has stints (index 3 & 4 are both for allisdo01 playing for 2 different teams) --> create 1 row per player
Appearances = Appearances.groupby(['playerID', 'yearID'])['G_c','G_1b','G_2b','G_3b','G_ss','G_of','G_dh'].sum()
Appearances.reset_index(inplace = True)
Appearances

<ipython-input-23-1686fe58b31c>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  Appearances = Appearances.groupby(['playerID', 'yearID'])['G_c','G_1b','G_2b','G_3b','G_ss','G_of','G_dh'].sum()


,playerID,yearID,G_c,G_1b,G_2b,G_3b,G_ss,G_of,G_dh
0,aardsda01,2004,0,0,0,0,0,0,0.0
1,aardsda01,2006,0,0,0,0,0,0,0.0
2,aardsda01,2007,0,0,0,0,0,0,0.0
3,aardsda01,2008,0,0,0,0,0,0,0.0
4,aardsda01,2009,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...
96607,zwilldu01,1915,0,3,0,0,0,148,0.0
96608,zwilldu01,1916,0,0,0,0,0,9,0.0
96609,zychto01,2015,0,0,0,0,0,0,0.0
96610,zychto01,2016,0,0,0,0,0,0,0.0


### Determining player position
* Some players may play in different positions in a year: Ex - 96607 plays 3 games as G_1b and 148 games as G_of in 1915.
* **Solution --> assume that the player position is the one that he played in most frequently during the season**

In [24]:
#Step1: Create a new variable which is the value of the number of appearances in the position played most frequently
Appearances["Max_G"] = Appearances[["G_c","G_1b","G_2b","G_3b","G_ss","G_of","G_dh"]].max(axis = 1)
Appearances

,playerID,yearID,G_c,G_1b,G_2b,G_3b,G_ss,G_of,G_dh,Max_G
0,aardsda01,2004,0,0,0,0,0,0,0.0,0.0
1,aardsda01,2006,0,0,0,0,0,0,0.0,0.0
2,aardsda01,2007,0,0,0,0,0,0,0.0,0.0
3,aardsda01,2008,0,0,0,0,0,0,0.0,0.0
4,aardsda01,2009,0,0,0,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
96607,zwilldu01,1915,0,3,0,0,0,148,0.0,148.0
96608,zwilldu01,1916,0,0,0,0,0,9,0.0,9.0
96609,zychto01,2015,0,0,0,0,0,0,0.0,0.0
96610,zychto01,2016,0,0,0,0,0,0,0.0,0.0


In [25]:
#Step2: Define the position name as the one which is the same as the most frequent
def Position(df):
    if (df["Max_G"] == df["G_c"]): return "C"
    elif (df["Max_G"] == df["G_1b"]): return "1B"
    elif (df["Max_G"] == df["G_2b"]): return "2B"
    elif (df["Max_G"] == df["G_3b"]): return "3B"
    elif (df["Max_G"] == df["G_ss"]): return "SS"
    elif (df["Max_G"] == df["G_of"]): return "OF"
    elif (df["Max_G"] == df["G_dh"]): return "DH"
    
#Apply the function to our dataset
Appearances["POS"] = Appearances.apply(Position, axis = 1)
# axis = 1: function applied row by row to create a new column
# axis = 0: function applied col by col to create a new row

Appearances

,playerID,yearID,G_c,G_1b,G_2b,G_3b,G_ss,G_of,G_dh,Max_G,POS
0,aardsda01,2004,0,0,0,0,0,0,0.0,0.0,C
1,aardsda01,2006,0,0,0,0,0,0,0.0,0.0,C
2,aardsda01,2007,0,0,0,0,0,0,0.0,0.0,C
3,aardsda01,2008,0,0,0,0,0,0,0.0,0.0,C
4,aardsda01,2009,0,0,0,0,0,0,0.0,0.0,C
...,...,...,...,...,...,...,...,...,...,...,...
96607,zwilldu01,1915,0,3,0,0,0,148,0.0,148.0,OF
96608,zwilldu01,1916,0,0,0,0,0,9,0.0,9.0,OF
96609,zychto01,2015,0,0,0,0,0,0,0.0,0.0,C
96610,zychto01,2016,0,0,0,0,0,0,0.0,0.0,C


In [26]:
Appearances = Appearances[Appearances["Max_G"] > 0]
Appearances = Appearances[["playerID", "yearID", "POS"]]
Appearances

,playerID,yearID,POS
9,aaronha01,1954,OF
10,aaronha01,1955,OF
11,aaronha01,1956,OF
12,aaronha01,1957,OF
13,aaronha01,1958,OF
...,...,...,...
96596,zuvelpa01,1991,3B
96605,zwilldu01,1910,OF
96606,zwilldu01,1914,OF
96607,zwilldu01,1915,OF


* **CREATE DUMMY VARIABLES**:
    * Catch = 1 if: POS = C
    * Infld = 1 if: POS = 2B or 3B or SS
    * 1B is the DEFAULT CATEGORY

In [27]:
Appearances["Catch"] = np.where(Appearances["POS"] == "C", 1, 0)
Appearances["Infld"] = np.where((Appearances["POS"] == "2B") | (Appearances["POS"] == "3B") | (Appearances["POS"] == "SS"),1,0)
Appearances

,playerID,yearID,POS,Catch,Infld
9,aaronha01,1954,OF,0,0
10,aaronha01,1955,OF,0,0
11,aaronha01,1956,OF,0,0
12,aaronha01,1957,OF,0,0
13,aaronha01,1958,OF,0,0
...,...,...,...,...,...
96596,zuvelpa01,1991,3B,0,1
96605,zwilldu01,1910,OF,0,0
96606,zwilldu01,1914,OF,0,0
96607,zwilldu01,1915,OF,0,0


In [28]:
Master = pd.merge(Master, Appearances, on = ["playerID", "yearID"], how = "left")
Master

,playerID,yearID,stint,G,AB,R,H,Doubles,Triples,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,PA,OBP,SLG,SalYear,teamID,lgID,salary,lnSal,debutyr,Exp,Arb,Free,POS,Catch,Infld
0,abbotje01,1998,1,89,244,33,68,14,1,12,41.0,3.0,3.0,9,28.0,1.0,0.0,2.0,5.0,2.0,260.0,0.298450,0.491803,1999,CHA,AL,255000,12.449019,1997,1,0,0,OF,0,0
1,abbotje01,2000,1,80,215,31,59,15,1,3,29.0,2.0,1.0,21,38.0,1.0,2.0,2.0,1.0,2.0,241.0,0.343096,0.395349,2001,FLO,NL,300000,12.611538,1997,3,1,0,OF,0,0
2,abbotku01,1994,1,101,345,41,86,17,3,9,33.0,3.0,0.0,16,98.0,1.0,5.0,3.0,2.0,5.0,371.0,0.290761,0.394203,1995,FLO,NL,119000,11.686879,1993,1,0,0,SS,0,1
3,abbotku01,1995,1,120,420,60,107,18,7,17,60.0,4.0,3.0,36,110.0,4.0,5.0,2.0,5.0,6.0,468.0,0.317597,0.452381,1996,FLO,NL,250000,12.429216,1993,2,0,0,SS,0,1
4,abbotku01,1996,1,109,320,37,81,18,7,8,33.0,3.0,3.0,22,99.0,1.0,3.0,4.0,0.0,7.0,349.0,0.307246,0.428125,1997,FLO,NL,650000,13.384728,1993,3,1,0,SS,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7779,zobribe01,2013,1,157,612,77,168,36,3,12,71.0,11.0,3.0,72,91.0,4.0,7.0,1.0,6.0,18.0,698.0,0.354376,0.401961,2014,TBA,AL,7000000,15.761421,2006,7,0,1,2B,0,1
7780,zobribe01,2014,1,146,570,83,155,34,3,10,52.0,10.0,5.0,75,84.0,4.0,1.0,2.0,6.0,8.0,654.0,0.354294,0.394737,2015,OAK,AL,7500000,15.830414,2006,8,0,1,2B,0,1
7781,zobribe01,2015,3,126,467,76,129,36,3,13,56.0,3.0,4.0,62,56.0,3.0,1.0,0.0,5.0,8.0,535.0,0.358879,0.449679,2016,CHN,NL,10500000,16.166886,2006,9,0,1,2B,0,1
7782,zuninmi01,2013,1,52,173,22,37,5,0,5,14.0,1.0,0.0,16,49.0,0.0,3.0,0.0,1.0,5.0,193.0,0.290155,0.329480,2014,SEA,AL,504100,13.130530,2013,0,0,0,C,1,0


# VI. Run the Regression

In [29]:
import statsmodels.formula.api as smf

### Regression 1: Covers years from 2000 to 2004

In [30]:
MB_Data = Master[(Master.SalYear >= 2000) & (Master.SalYear <= 2004)]

In [31]:
Val_All_lm = smf.ols(formula = "lnSal ~ OBP + SLG + PA + Arb + Free + Catch + Infld", data = MB_Data).fit()
Val_All_lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lnSal   R-squared:                       0.662
Model:                            OLS   Adj. R-squared:                  0.661
Method:                 Least Squares   F-statistic:                     485.7
Date:                Sun, 12 Dec 2021   Prob (F-statistic):               0.00
Time:                        14:09:21   Log-Likelihood:                -1928.6
No. Observations:                1741   AIC:                             3873.
Df Residuals:                    1733   BIC:                             3917.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.1090      0.164     61.773      0.000       9.788      10.430
OBP            1.4783      0.628      2.355      0.019       0.247       2.709
SLG            2.3884      0.316      7.553      0.000       1.768       3.009
PA             0.0031      0.000     25.962      0.000       0.003       0.003
Arb            1.2076      0.049     24.585      0.000       1.111       1.304
Free           1.8050      0.049     37.033      0.000       1.709       1.901
Catch          0.1115      0.057      1.972      0.049       0.001       0.222
Infld         -0.0502      0.041     -1.231      0.219      -0.130       0.030
==============================================================================
Omnibus:                        5.756   Durbin-Watson:                   1.493
Prob(Omnibus):                  0.056   Jarque-Bera (JB):                6.893
Skew:                           0.026   Prob(JB):                       0.0319
Kurtosis:                       3.304   Cond. No.                     1.84e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.84e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

* Analyse Result:
    * Size of coeff of SLG is almost double of OBP --> **Until 2004, slugging percentage was more significant than on-base percentage while determing the player's salary**

### Regression 2: Covers years from 2000 - 2003

In [32]:
MB_Data_Pre = Master[(Master.SalYear >= 2000) & (Master.SalYear <= 2003)]
Val_Pre_lm = smf.ols(formula = "lnSal ~ OBP + SLG + PA + Arb + Free + Catch + Infld", data = MB_Data_Pre).fit()
Val_Pre_lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  lnSal   R-squared:                       0.681
Model:                            OLS   Adj. R-squared:                  0.679
Method:                 Least Squares   F-statistic:                     423.2
Date:                Sun, 12 Dec 2021   Prob (F-statistic):               0.00
Time:                        14:09:23   Log-Likelihood:                -1512.2
No. Observations:                1398   AIC:                             3040.
Df Residuals:                    1390   BIC:                             3082.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.2248      0.176     58.008      0.000       9.879      10.571
OBP            0.7958      0.677      1.176      0.240      -0.532       2.123
SLG            2.4908      0.340      7.322      0.000       1.823       3.158
PA             0.0031      0.000     24.172      0.000       0.003       0.003
Arb            1.2559      0.054     23.432      0.000       1.151       1.361
Free           1.8729      0.053     35.018      0.000       1.768       1.978
Catch          0.1330      0.062      2.156      0.031       0.012       0.254
Infld         -0.0282      0.044     -0.639      0.523      -0.115       0.058
==============================================================================
Omnibus:                        7.962   Durbin-Watson:                   1.499
Prob(Omnibus):                  0.019   Jarque-Bera (JB):               10.336
Skew:                          -0.041   Prob(JB):                      0.00570
Kurtosis:                       3.413   Cond. No.                     1.82e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.82e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Regression 3: Year by Year from 2000 to 2004

In [33]:
MB_Data_2000 = Master[(Master.SalYear == 2000)]
Val_2000_lm = smf.ols(formula = "lnSal ~ OBP + SLG + PA + Arb + Free + Catch + Infld", data = MB_Data_2000).fit()
MB_Data_2001 = Master[(Master.SalYear == 2001)]
Val_2001_lm = smf.ols(formula = "lnSal ~ OBP + SLG + PA + Arb + Free + Catch + Infld", data = MB_Data_2001).fit()
MB_Data_2002 = Master[(Master.SalYear == 2002)]
Val_2002_lm = smf.ols(formula = "lnSal ~ OBP + SLG + PA + Arb + Free + Catch + Infld", data = MB_Data_2002).fit()
MB_Data_2003 = Master[(Master.SalYear == 2003)]
Val_2003_lm = smf.ols(formula = "lnSal ~ OBP + SLG + PA + Arb + Free + Catch + Infld", data = MB_Data_2003).fit()
MB_Data_2004 = Master[(Master.SalYear == 2004)]
Val_2004_lm = smf.ols(formula = "lnSal ~ OBP + SLG + PA + Arb + Free + Catch + Infld", data = MB_Data_2004).fit()

In [34]:
from statsmodels.iolib.summary2 import summary_col
Header = ["All years", "2000-2003", "2000", "2001", "2002", "2003", "2004"]
Table_3 = summary_col([Val_All_lm, Val_Pre_lm, Val_2000_lm, Val_2001_lm, Val_2002_lm, Val_2003_lm, Val_2004_lm],
                     regressor_order = ['OBP', 'SLG', 'PA', 'Arb', 'Free', 'Catch', 'Infld', 'Intercept'], stars = True,
                     float_format = "'%.3f'", model_names = Header)
print(Table_3)


                All years   2000-2003     2000        2001        2002        2003       2004   
------------------------------------------------------------------------------------------------
OBP            '1.478'**   '0.796'     '2.179'*   '0.132'     '0.595'     '1.890'     '4.353'***
               ('0.628')   ('0.677')   ('1.217')  ('1.209')   ('1.566')   ('1.585')   ('1.630') 
SLG            '2.388'***  '2.491'***  '2.550'*** '3.224'***  '2.312'***  '1.945'**   '2.171'***
               ('0.316')   ('0.340')   ('0.610')  ('0.597')   ('0.724')   ('0.846')   ('0.820') 
PA             '0.003'***  '0.003'***  '0.002'*** '0.003'***  '0.003'***  '0.003'***  '0.003'***
               ('0.000')   ('0.000')   ('0.000')  ('0.000')   ('0.000')   ('0.000')   ('0.000') 
Arb            '1.208'***  '1.256'***  '1.296'*** '1.111'***  '1.289'***  '1.247'***  '1.044'***
               ('0.049')   ('0.054')   ('0.102')  ('0.100')   ('0.114')   ('0.114')   ('0.120') 
Free           '1.805'***  '1

# VII. Analyse Results of the Table

* The standard error for SLG is always statistically significant (it is always: p<0.05)  --> **SLG is consistently significant in statistical sense**
* The standard error for OBP is always statistically insignificant, untill 2014 --> **Before 2014, OBP was never statistically significant while determing player salary**
* The coefficient of OBP almost doubles from 2003 to 2004 --> **2004 is when the moneyball book was published! It had a huge impact on the importance given to OBP while determing player salaries**
* Until 2004, the coeff of SLG was always bigger than that of OBP. This changed in 2004 --> **From 2004 onwards, OBP became more significant while determing player salary, than SLG**